In [1]:
import os
import json

h5_directory = r'data/Dreamer/h5'
import torch
import json

from dosed.datasets import BalancedEventDataset as dataset
from dosed.datasets import get_train_validation_test

train, validation, test = get_train_validation_test(h5_directory,
                                                    percent_test=25,
                                                    percent_validation=33,
                                                    seed=0)
window = 3# 3s
radio_positive = 0.5 # 一个训练批次中，每个样本包含至少一个脑电活动（spindle）的概率
fs = 32 # 原始64hz，降采样到32hz
signals = [
    {
        'h5_path': '/eeg_0',
        'fs': 64,
        'processing': {
            "type": "clip_and_normalize",
            "args": {
                    "min_value": -150,
                "max_value": 150,
            }
        }
    },
    {
        'h5_path': '/eeg_1',
        'fs': 64,
        'processing': {
            "type": "clip_and_normalize",
            "args": {
                    "min_value": -150,
                "max_value": 150,
            }
        }
    }
]

events = [
    {
        "name": "spindle",
        "h5_path": "spindle",
    },
]

dataset_parameters = {
    "h5_directory": h5_directory,#h5目录
    "signals": signals,#信号，包括两个通道
    "events": events,#事件，只有一个事件，即脑电活动（spindle）
    "window": window,#窗口
    "fs": fs,#采样率32
    #"ratio_positive": ratio_positive,#一个训练批次中，每个样本包含至少一个脑电活动（spindle）的概率
    "n_jobs": -1,  # Make use of parallel computing to extract and normalize signals from h5
    "cache_data": True,  # by default will store normalized signals extracted from h5 in h5_directory + "/.cache" directory
}
train
validation
test

c:\Users\chenjinghuang\anaconda3\envs\TS-TCC\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


['68749217-a3e7-41ff-b965-53293b1a6147.h5',
 'd32fbf45-7bcf-4804-9f20-34823de54bcf.h5',
 'b33632ff-3648-4604-93e2-c6313b16a09e.h5',
 'a6624e57-c003-4c32-8ac5-03fc5770ccf8.h5',
 '0e20159f-cd04-4ce8-afaf-687dd450ba73.h5']

In [2]:
dataset_train = dataset(records=train, **dataset_parameters)
dataset_validation = dataset(records=validation, **dataset_parameters)
dataset_test = dataset(records=test, **dataset_parameters)

In [3]:
def save_dataset(dataset, filename):
    data_list = []
    label_list = []
    for i in range(len(dataset)):
        data_list.append(dataset[i][0])
        if dataset[i][1].nelement() != 0:
            label_list.append(1)
        else:
            label_list.append(0)
    data_tensor = torch.stack(data_list)
    label_tensor = torch.tensor(label_list)
    data_dict = {'samples': data_tensor, 'labels': label_tensor}
    torch.save(data_dict, filename)

In [4]:
save_dataset(dataset_train, 'train.pt')
save_dataset(dataset_validation, 'validation.pt')
save_dataset(dataset_test, 'test.pt')


d:\Desktop\research\github repo\CA-TCC\dosed\datasets\dataset.py:388: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:141.)
  return torch.FloatTensor(signal_data), torch.FloatTensor(events_data)#相对开始时间、相对结束时间和标签添加到 events_data 中
